In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
try:
  del model
except NameError:
  pass

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [5]:
#Un-comment this section to download the dataset
'''
import urllib.request

print('Beginning file download with urllib2...')

url = 'http://datax.kennesaw.edu/imdb_wiki/imdb4p5.csv'  
urllib.request.urlretrieve(url, 'drive/data/imdb.csv')
'''

"\nimport urllib.request\n\nprint('Beginning file download with urllib2...')\n\nurl = 'http://datax.kennesaw.edu/imdb_wiki/imdb4p5.csv'  \nurllib.request.urlretrieve(url, 'drive/data/imdb.csv')\n"

In [0]:
data = pd.read_csv('drive/data/imdb4p5.csv')
#data = pd.read_csv('drive/data/wiki5.csv')

In [7]:
data.head()
#data.shape

,Unnamed: 0,gender,age,px0,px1,px2,px3,px4,px5,px6,...,px9990,px9991,px9992,px9993,px9994,px9995,px9996,px9997,px9998,px9999
0,0,0,49,16,15,17,18,15,15,15,...,11,23,193,132,11,5,12,12,14,14
1,1,0,81,47,48,49,47,49,51,51,...,33,33,33,33,33,33,33,33,33,32
2,2,0,81,19,19,19,19,19,19,19,...,8,8,8,7,6,5,8,4,227,217
3,3,0,80,0,0,0,0,0,0,0,...,36,59,39,19,9,27,25,24,24,24
4,4,1,37,184,191,199,201,195,197,198,...,230,228,226,226,224,221,217,217,211,205


In [0]:
df_x = data.iloc[:,3:].values.reshape(len(data),100,100,1)

y = data.iloc[:,1].values

del data

In [0]:
df_y = keras.utils.to_categorical(y,num_classes=2)

In [0]:
df_x = np.array(df_x)
df_y = np.array(df_y)

In [11]:
df_x.shape

(33147, 100, 100, 1)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df_x,df_y,test_size=0.3,random_state=4)

del df_x
del df_y

print(x_train.shape)
print(x_test.shape)

(23202, 100, 100, 1)
(9945, 100, 100, 1)


In [0]:
#CNN MODEL

model = Sequential()
model.add(Convolution2D(64,3,data_format='channels_last',activation='relu',input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32,3,data_format='channels_last',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(16,3,data_format='channels_last',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(8,3,data_format='channels_last',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(4,3,data_format='channels_last',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 8)           1160      
__________

In [15]:
#model.fit(x_train[:2001], y_train[:2001], validation_data=(x_test[:21], y_test[:21]), epochs=5)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=200)

Train on 23202 samples, validate on 9945 samples
Epoch 1/20
23202/23202 [==============================] - 26s 1ms/step - loss: 0.7386 - acc: 0.5577 - val_loss: 0.5969 - val_acc: 0.6968
Epoch 2/20
23202/23202 [==============================] - 20s 876us/step - loss: 0.5151 - acc: 0.7553 - val_loss: 0.4677 - val_acc: 0.7826
Epoch 3/20
23202/23202 [==============================] - 20s 879us/step - loss: 0.3592 - acc: 0.8503 - val_loss: 0.2969 - val_acc: 0.8789
Epoch 4/20
23202/23202 [==============================] - 20s 878us/step - loss: 0.2642 - acc: 0.8974 - val_loss: 0.2181 - val_acc: 0.9139
Epoch 5/20
23202/23202 [==============================] - 20s 880us/step - loss: 0.2024 - acc: 0.9248 - val_loss: 0.1801 - val_acc: 0.9333
Epoch 6/20
23202/23202 [==============================] - 20s 882us/step - loss: 0.1684 - acc: 0.9402 - val_loss: 0.3623 - val_acc: 0.8450
Epoch 7/20
23202/23202 [==============================] - 20s 876us/step - loss: 0.1435 - acc: 0.9503 - val_loss: 0.145

In [16]:
#np.savez('drive/data/X',x_train,x_test,y_train,y_test)
model.evaluate(x_test,y_test)

9945/9945 [==============================] - 4s 451us/step


[0.12759868322514362, 0.957968828503123]

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict_classes(x_test)
print(classification_report(np.argmax(y_test,axis=1),y_pred))

             precision    recall  f1-score   support

          0       0.96      0.97      0.96      5656
          1       0.96      0.94      0.95      4289

avg / total       0.96      0.96      0.96      9945



In [19]:
print(confusion_matrix(np.argmax(y_test,axis=1),y_pred))

[[5478  178]
 [ 240 4049]]
